In [1]:
#Import packages
import collections
import numpy as np
import itertools
import os
import shutil
import glob
import csv
import pandas as pd

In [5]:
def clean(words):
    out = []
    for c in words:
        c = c.lower()
        if c in set('abcdefghijklmnopqrstuvwxyz '):
            out.append(c)
    return ''.join(out)

#Construct sliding window
wordlist=[]
for path in glob.glob(os.path.join(os.getcwd(),'train_error','*.txt')):
    with open(path, encoding="utf8") as file:
        for line in file:
            line=clean(line)
            for words in line.split():
                wordlist.append(words)       
#sliding window   
sliding_list=[]       
for i in range(2,len(wordlist)-2,1):
    sliding_list.append(wordlist[i-2:i+3])

flat_list=[]
for sublist in sliding_list:
    for item in sublist:
        flat_list.append(item)
df = pd.DataFrame(np.array(flat_list).reshape(len(sliding_list),5), columns = list("abcde"))
df["tess"]=df["c"]
df.shape

(191027, 6)

In [3]:
#Generate error & truth one-to-one matching dataframe
list_a=[]
list_b=[] 
equal=[]
combine=pd.DataFrame()
for path1,path2 in zip(glob.glob(os.path.join(os.getcwd(),'train_error','*.txt')),glob.glob(os.path.join(os.getcwd(),'train_truth','*.txt'))):
    with open(path1, encoding="utf8") as file1:
        with open(path2, encoding="utf8") as file2:
            for line1,line2 in zip(file1,file2):
                line1=clean(line1)
                line2=clean(line2)
                if len(line1.split())==len(line2.split()):
                    equal.append(1)
                    for word1,word2 in zip(line1.split(),line2.split()): 
                        list_a.append(word1)
                        list_b.append(word2)
                else:
                    equal.append(0)
                    add=list(np.repeat("Truuuuuuuuuuue",len(line1.split())))
                    list_a=list_a+add
                    list_b=list_b+add
list_b=list_b[2:]
list_b=list_b[:len(list_b)-2]
df["Truth"]=list_b
df = df[df.Truth != "Truuuuuuuuuuue"]

In [4]:
##Detecting error terms
#Combine all train_set files
read_files = glob.glob(os.path.join(os.getcwd(), 'train_truth', "*.txt"))
with open("train_truth.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())
#Find error terms
file_truth=[]          
with open('train_truth.txt', encoding="utf8") as file1: 
    for line in file1:
        for words in line.split():
            file_truth.append(words)

detect_list=[]
my_list = df["tess"].values
for i in range(len(my_list)):
    if my_list[i] in file_truth:
        detect_list.append("1")
    else:
        detect_list.append("0")
        
se = pd.Series(detect_list)
df['delete'] = se.values
#Write out
df = df[df.delete != "1"]
writer = pd.ExcelWriter('Detection_clean_test.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()